In [21]:
import pandas as pd
orders_table = pd.read_csv("../data/olist_orders_dataset.csv")
order_items_table = pd.read_csv("../data/olist_order_items_dataset.csv")

# 1.去除异常数据 
"""
    (2016, 9),   # 2016年9月
    (2016, 10),  # 2016年10月
    (2016, 12),  # 2016年12月
    (2018, 9),   # 2018年9月
    (2018, 10)   # 2018年10月
"""
orders_table['order_purchase_timestamp'] = pd.to_datetime(orders_table['order_purchase_timestamp'],errors='coerce')
orders_table['year'] = orders_table['order_purchase_timestamp'].dt.year
orders_table['month'] = orders_table['order_purchase_timestamp'].dt.month
periods_to_remove = [
    (2016, 9),
    (2016, 10),  # 2016年10月
    (2016, 12),  # 2016年12月
    (2018, 9),   # 2018年9月
    (2018, 10)   # 2018年10月
]
delete_condition = False
for year, month in periods_to_remove:
    delete_condition |= ((orders_table['year'] == year) & (orders_table['month'] == month))
    # 删除指定时间段的数据
orders_table = orders_table[~delete_condition]

# 额外异常检查 由于不存在异常金额（负数金额）后续不进行处理
anomalies = order_items_table[(order_items_table['price']<=0) | (order_items_table['freight_value']<0)]
print("\n 异常金额记录数（price<=0 或 freight<0）：", len(anomalies))



 异常金额记录数（price<=0 或 freight<0）： 0


In [22]:
# 2.检查金额异常值
for col in ['price','freight_value']:
    desc = order_items_table[col].describe(percentiles=[.01,.25,.5,.75,.95,.99])
    print(f"\n {col} 分布统计：\n", desc)

# 2.1去掉极端值
p99_price = order_items_table['price'].quantile(0.99)
order_items_table.loc[order_items_table['price'] > p99_price, 'price'] = p99_price
p99_freight_value = order_items_table['freight_value'].quantile(0.99)
order_items_table.loc[order_items_table['freight_value'] > p99_freight_value, 'freight_value'] = p99_freight_value


 price 分布统计：
 count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
1%            9.990000
25%          39.900000
50%          74.990000
75%         134.900000
95%         349.900000
99%         890.000000
max        6735.000000
Name: price, dtype: float64

 freight_value 分布统计：
 count    112650.000000
mean         19.990320
std          15.806405
min           0.000000
1%            4.419800
25%          13.080000
50%          16.260000
75%          21.150000
95%          45.120000
99%          84.520000
max         409.680000
Name: freight_value, dtype: float64


In [23]:
# 3.检查关联完整性
missing_orders = order_items_table[~order_items_table['order_id'].isin(orders_table['order_id'])]
missing_items = orders_table[~orders_table['order_id'].isin(order_items_table['order_id'])]
# items无订单
print("\n items中找不到对应订单的数量：", len(missing_orders))
if len(missing_orders) > 0:
    order_items_table = order_items_table[order_items_table['order_id'].isin(orders_table['order_id'])]
# 订单无明细
print(" orders中没有明细的订单数量：", len(missing_items))
if len(missing_items) > 0:
    orders_table = orders_table[orders_table['order_id'].isin(order_items_table['order_id'])]



 items中找不到对应订单的数量： 371
 orders中没有明细的订单数量： 739


In [24]:
missing_orders

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
908,02190241f7190a1f3c7e0df95a749c6a,1,e00d2c49504aeaf7acdc9ff7cf8a6107,46dc3b2cc0980fb8ec44634e21d2718e,2016-10-12 04:44:23,249.99,10.74
909,02190241f7190a1f3c7e0df95a749c6a,2,e00d2c49504aeaf7acdc9ff7cf8a6107,46dc3b2cc0980fb8ec44634e21d2718e,2016-10-12 04:44:23,249.99,10.74
911,021d08e4057e5b6414c8d2f69f4f72a0,1,a819714dd23a6ac18b1be2b13686017b,ffff564a4f9085cd26170f4732393726,2016-10-17 19:01:41,51.50,14.53
1138,02a0eb7c22b0616c767a45954a2a28f6,1,5f8c74f4a3d2b616857fcb7d4dd1f61a,83e197e95a1bbabc8c75e883ed016c47,2016-10-14 08:14:53,119.50,25.04
1320,03128233e78ed8ade6738f2043f4cf8d,1,3cacd5299c846cb08ae1e445fd78392b,c7dcd301ecfe5ab7f778ac172cf74be7,2016-10-21 16:25:24,57.90,14.61
...,...,...,...,...,...,...,...
109873,f9b03f7a6e5788ffb6cb6b4cd0d01892,1,44377a26f7c80dbf0a9b2b300604e6bf,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-10-15 05:02:52,99.90,16.27
109906,f9c44da06151c190a9a6c9c712873d10,1,b864b103180d5a6a3f830e67f93e8735,f5b84683a9bf9e1df748cf40f601b39c,2016-10-15 15:09:06,83.90,9.07
110607,fb5114c39f632a52598cf770c6076620,1,269e87010f6d5e57015a90b00983e334,0db783cfcd3b73998abc6e10e59a102f,2016-10-13 14:35:36,35.00,16.05
110800,fbbc9a4fa88f1ccd651790df665b3e57,1,44377a26f7c80dbf0a9b2b300604e6bf,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-10-09 02:45:17,99.90,19.53


In [25]:
missing_items['order_status'].value_counts()


order_status
unavailable    602
canceled       132
created          5
Name: count, dtype: int64

In [26]:

# 4.验证订单状态分布
status_check = orders_table['order_status'].value_counts(normalize=True).mul(100).round(2)
status_count = orders_table['order_status'].value_counts()
status_dis = pd.DataFrame({
    "count":status_count,
    "pct":status_check
})
print("订单状态分别情况",status_dis)

# 5.筛选delivered订单
cond1 = orders_table['order_status'] == 'delivered'
orders  =  orders_table.loc[cond1,:]
orders_table =  orders_table.loc[cond1,['order_id','order_purchase_timestamp','order_delivered_customer_date']]



订单状态分别情况               count    pct
order_status              
delivered     96211  97.82
shipped        1097   1.12
canceled        448   0.46
processing      299   0.30
invoiced        296   0.30
approved          2   0.00


In [27]:
order_items_table.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [28]:
# 7.构建gmv_value字段
# order_items_table['gmv_value'] = order_items_table['price'] + order_items_table['freight_value']
items = order_items_table.copy()
order_items = order_items_table[['order_id','product_id','price','freight_value']]
order_item_wide = orders_table.merge(order_items,on='order_id',how='inner')

order_item_wide = order_item_wide.loc[:,['order_id','product_id','order_purchase_timestamp','order_delivered_customer_date','price','freight_value']]
order_item_wide.to_csv("../processed/order_item_wide_gmv_analysis2.csv",index=False)